In [17]:
import itertools
import numpy as np

from docplex.mp.model import Model
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import CplexOptimizer
from qiskit_optimization.translators import from_docplex_mp
from qiskit.opflow import *

In [18]:
t = np.arange(1, 4)

mu = -2 * t
sigma = np.outer(t, t)

def cost(x):
    return x.T.dot(sigma).dot(x) + x.dot(mu)

In [21]:
for x in itertools.product([0, 1], repeat=t.size):
    print(x, cost(np.asarray(x)))

(0, 0, 0) 0
(0, 0, 1) 3
(0, 1, 0) 0
(0, 1, 1) 15
(1, 0, 0) -1
(1, 0, 1) 8
(1, 1, 0) 3
(1, 1, 1) 24


In [11]:
def create_problem(mu: np.array, sigma: np.array) -> QuadraticProgram:
    """Solve the quadratic program using docplex."""

    mdl = Model()
    x = [mdl.binary_var("x%s" % i) for i in range(len(sigma))]

    objective = mdl.sum([mu[i] * x[i] for i in range(len(mu))])
    objective += mdl.sum(
        [sigma[i, j] * x[i] * x[j] for i in range(len(mu)) for j in range(len(mu))]
    )
    mdl.minimize(objective)

    qp = from_docplex_mp(mdl)
    return qp

qubo = create_problem(mu, sigma)

In [12]:
qubo

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex_model3

Minimize
 obj: - 2 x0 - 4 x1 - 6 x2 + [ 2 x0^2 + 8 x0*x1 + 12 x0*x2 + 8 x1^2 + 24 x1*x2
      + 18 x2^2 ]/2
Subject To

Bounds
 0 <= x0 <= 1
 0 <= x1 <= 1
 0 <= x2 <= 1

Binaries
 x0 x1 x2
End

In [14]:
ref = CplexOptimizer().solve(qubo)
print(ref)

optimal function value: -1.0
optimal value: [1. 0. 0.]
status: SUCCESS
